In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define the directory containing the datasets
data_dir = "../../data/"
dataset_path = os.path.join(data_dir, "analyzed/catHarmQA/combined_catqa.csv")

data = pd.read_csv(dataset_path)
data.shape

(136400, 19)

In [7]:
data.columns

Index(['category', 'subcategory', 'original_question',
       'original_question_safety', 'original_response',
       'original_response_safety', 'original_response_pre',
       'original_response_pre_safety', 'experiment', 'perturbation_level',
       'perturbation_type', 'perturbation_count', 'perturbed_question',
       'perturbed_question_safety', 'model', 'perturbed_response',
       'perturbed_response_safety', 'perturbed_response_pre',
       'perturbed_response_pre_safety'],
      dtype='object')

Text in "orignal_response" column start with question which from "original_question" columns and It's not needed, so we have to remove that question. so, we check if question from "original_question" match with start portion of "orignal_response" then remove that question from "orignal_response" column. same will happen with "perturbed_response" column which have question from "perturbed_question" column. remember that data variable is pandas dataframe and we can access columns by their names.

In [252]:
def handle_eage_case(content):
    # edge cases
    for condition in [" ' ", " '", " !", " ,", " ?", " .", " '"]:
        if condition in content:
            content = content.replace(condition, condition.strip())
        # content = re.sub(r"\s*'\s*", "'", content)

    return content


# Function to remove question from response
def remove_question_from_response(row, question_col, response_col, handle_edge_case_condition=False):
    question = row[question_col] if not handle_edge_case_condition else handle_eage_case(row[question_col])
    response = row[response_col] if not handle_edge_case_condition else handle_eage_case(row[response_col])
    if response.startswith(question):
        return response[len(question) :].strip()
    else:
        if handle_edge_case_condition and not response.startswith(question):
            print(f"{row.name}th row")
            print(f"question: {question}")
            print(f"response: {response}", f"\n{'-'*120}\n")      
            return response
        else:
            return remove_question_from_response(row, question_col, response_col, True)

In [253]:
# Apply the function to both original and perturbed columns
data["original_response_pre"] = data.apply(
    lambda row: remove_question_from_response(
        row, "original_question", "original_response"
    ),
    axis=1,
)
data["perturbed_response_pre"] = data.apply(
    lambda row: remove_question_from_response(
        row, "perturbed_question", "perturbed_response"
    ),
    axis=1,
)

Now let's test if prefix question is removed from "original_response" and "perturbed_response" columns. we will do that by coding. we will take difference between "original_response_pre" and "original_response", it will output text from "original_response" which is not in "original_response_pre" and different will be equal to "original_question". so perform this check over all rows and return count of False. we will do same for "perturbed_response_pre" and "perturbed_response" columns.

In [254]:
def find_non_common_substring(A, B):
    index = A.find(B)
    if index == -1:
        return A
    else:
        return A[:index]

# Function to check if the prefix question is removed
def check_prefix_removed(row, question_col, response_pre_col, response_col, handle_edge_case_condition=False):
    question = row[question_col] if not handle_edge_case_condition else handle_eage_case(row[question_col])
    response = row[response_col] if not handle_edge_case_condition else handle_eage_case(row[response_col])
    response_pre = row[response_pre_col]

    non_common_substring = find_non_common_substring(response, response_pre).strip()

    if non_common_substring.strip() == question.strip():
        return True
    else:
        if handle_edge_case_condition and non_common_substring.strip() != question.strip():
            # print(f"{row.name}th row")
            # print(f"question: {question}")
            # print(f"response: {response}")
            # print(f"resp_pre: {response_pre}", f"\n{'-'*120}\n")
            return question.__contains__(non_common_substring)
        else:
            return check_prefix_removed(row, question_col, response_pre_col, response_col, True)

In [255]:
# Apply the check function to both original and perturbed columns
original_check = data.apply(
    lambda row: check_prefix_removed(
        row, "original_question", "original_response_pre", "original_response"
    ),
    axis=1,
)
perturbed_check = data.apply(
    lambda row: check_prefix_removed(
        row, "perturbed_question", "perturbed_response_pre", "perturbed_response"
    ),
    axis=1,
)

## Check and Validation of above Technique

In [256]:
# Check if all rows satisfy `original_question + original_response_pre == original_response`
original_check.all()

True

In [257]:
# Check if all rows satisfy `perturbed_question + perturbed_response_pre == perturbed_response`
perturbed_check.all()

True

In [258]:
perturbed_false_count = len(perturbed_check) - perturbed_check.sum()
print(f"Count of False for perturbed_response: {perturbed_false_count}")

Count of False for perturbed_response: 0


In [259]:
perturbed_check[perturbed_check == False]

Series([], dtype: bool)

In [260]:
print(
    f"perturbed_response_pre == perturbed_response: \n{perturbed_check[perturbed_check == False].index.to_series().apply(lambda x: data.loc[x, "perturbed_response_pre"] == data.loc[x, "perturbed_response"]).value_counts()}"
)

perturbed_response_pre == perturbed_response: 
Series([], Name: count, dtype: int64)


In [261]:
perturbed_check[perturbed_check == False].index.to_series().apply(lambda x: data.loc[x, "model"]).value_counts()

Series([], Name: count, dtype: int64)

In [262]:
for index in perturbed_check[perturbed_check == False].index:
    print("------ ", data.loc[index, "perturbed_question"], "\n")
    print("------ ", data.loc[index, "perturbed_response"], "\n")
    print("------ ", data.loc[index, "perturbed_response_pre"], "\n")
    print(data.loc[index, "perturbed_response_pre"] == data.loc[index, "perturbed_response"], "\n")
    print(f"{'-'*120}\n")

In [267]:
# data.to_csv(dataset_path, index=False)

## preprocess safety response

In [12]:
dataset_path_1 = os.path.join(data_dir, "analyzed/catHarmQA/combined_catqa1.csv")
dataset_path_2 = os.path.join(data_dir, "analyzed/catHarmQA/combined_catqa2.csv")

data1 = pd.read_csv(dataset_path_1)
data2 = pd.read_csv(dataset_path_2)

# for column in ['category', 'subcategory', 'original_question', 'original_question_safety', 'original_response', 'original_response_safety', 'perturbation_level', 'perturbation_type', 'perturbed_question', 'perturbed_question_safety', 'model', 'perturbed_response', 'perturbed_response_safety', 'experiment', 'original_response_pre', 'original_response_pre_safety', 'perturbed_response_pre', 'perturbed_response_pre_safety']:
#     data1[column] = data1[column].astype('string')
#     data2[column] = data2[column].astype('string')

# data1.to_csv(dataset_path_1, index=False)
# data2.to_csv(dataset_path_2, index=False)

## Analysis - After removing questions from response

In [13]:
data2.columns

Index(['category', 'subcategory', 'original_question',
       'original_question_safety', 'original_response',
       'original_response_safety', 'perturbation_level', 'perturbation_type',
       'perturbation_count', 'perturbed_question', 'perturbed_question_safety',
       'model', 'perturbed_response', 'perturbed_response_safety',
       'experiment', 'original_response_pre', 'perturbed_response_pre'],
      dtype='object')

In [11]:
data2.to_csv(dataset_path_2, index=False)

In [5]:
data.perturbed_response_pre_safety.value_counts()

perturbed_response_pre_safety
safe      92586
unsafe    43814
Name: count, dtype: int64

In [14]:
changed_rows = data[
    data["perturbed_response_safety"] != data["perturbed_response_pre_safety"]
]
num_changed = changed_rows.shape[0]
print(f"Number of changed rows: {num_changed}")

Number of changed rows: 59206


In [15]:
percentage_changed = (num_changed / len(data)) * 100
print(f"Percentage of changed rows: {percentage_changed:.2f}%")

Percentage of changed rows: 43.41%


In [3]:
data.original_question_safety.value_counts(normalize=True)

original_question_safety
unsafe    0.932727
safe      0.067273
Name: proportion, dtype: float64